In [8]:
import pandas as pd
import seaborn as sns
import itertools
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

In [55]:
df_1bed_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_1_bed_flat_monetary_2013-2021.csv')
bed_2_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_flat_monetary_2013-2021.csv')
bed_2_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_house_monetary_2013-2021.csv')
bed_3_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_flat_monetary_2013-2021.csv')
bed_3_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_house_monetary_2013-2021.csv')
bed_4_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_4_house_monetary_2013-2021.csv')

In [75]:
df_1bed_flat_no_covid =pd.read_csv('../data/curated/Rental_Price_historical/joined_1_bed_flat_monetary_2013-2019.csv')
bed_2_flat_no_covid=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_flat_monetary_2013-2019.csv')
bed_2_house_no_covid=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_house_monetary_2013-2019.csv')
bed_3_flat_no_covid=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_flat_monetary_2013-2019.csv')
bed_3_house_no_covid=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_house_monetary_2013-2019.csv')
bed_4_house_no_covid=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_4_house_monetary_2013-2019.csv')

# Prediction
The features are:
1. Median_age_persons 
2. Median_mortgage_repay_monthly
3. Median_tot_prsnl_inc_weekly
4. Average_household_size
5. Population
6. Alleged Offender Incident Rate
7. Victimization Rate
8. Offender Count

In [56]:
def get_forecast_all_suburbs(raw_df, yrs, PRED_COL):
    # cols_imputed = {}
    forecasted_data = {}
    for index, row in raw_df.iterrows():
        X_df = make_2D(row, yrs, PRED_COL)


        forecasted_data[row['Suburb']] = forecast(X_df)


    return forecasted_data

def make_2D(suburb_row, yrs, PRED_COL):
    cols_imputed = {}
    X_df = {}
    for feature in PRED_COL:
        f = []
        for t in suburb_row.iteritems():
            if feature in t[0]:
                f.append(t[1])
        X_df[feature] = f

    X_df = pd.DataFrame.from_dict(X_df)
    X_df.loc[:,'year'] = yrs
    X_df.rename(columns = {"Median_Mar":"Median_rental_price"}, inplace = True)
    
    for col in X_df:
        year = []
        data = []
        if col != 'year':
            for i in range(0,len(yrs)):
                if (X_df.loc[i,col] != -1):
                    data.append(X_df.loc[i,col])
                    year.append(yrs[i]) 
            if (len(year) == 0):
                continue   
            df = pd.DataFrame()     
            df["year"]=year
            df["data"]=data 
            reg = LinearRegression().fit(df[["year"]], df["data"])
            
            arr = []
            for yr in yrs:
                arr.extend(reg.predict(np.array([[yr]])))
    #                     print(arr)
        cols_imputed[col] = arr


    for i in range(0,len(yrs)):
        for col in cols_imputed:
            if (X_df.loc[i,col] == -1):
                X_df.loc[i,col] = cols_imputed[col][i]

    return X_df


def forecast(X_df):
    future_yrs = [2022,2023,2024,2025]
    reg_forecast = pd.DataFrame()
    for col in X_df.iloc[:, 1:-1]:
        l = []
        reg = LinearRegression().fit(X_df['year'].values.reshape((-1,1)), X_df[col].values.reshape((-1,1)))
        for yr in future_yrs:
            l.extend(reg.predict(np.array([[yr]])))
            row = [np.round(x[0], 1) for x in l]
    #         print(l)
        reg_forecast[col] = row
    reg_forecast['year'] = future_yrs
    regr = make_pipeline(StandardScaler(), SVR(kernel='poly', C=3, epsilon=0.2))

    regr.fit(X_df.iloc[:,[1,2,3,7,8,9,10,11]], X_df['Median_rental_price'])
    future_price = regr.predict(reg_forecast.iloc[:,[0,1,2,6,7,8,9,10]])

    
    
#     print(reg_forecast.iloc[:,[0,1,2,6]])
    # Median_age_persons, Median_mortgage_repay_monthly, Median_tot_prsnl_inc_weekly, Average_household_size
    
    reg_forecast["Median_rental_price"] = future_price
    # reg = LinearRegression().fit(X_df.iloc[:,[1,2,3,7]], X_df['Median_rental_price'])

    # future_price = reg.predict(reg_forecast.iloc[:,[0,1,2,6]])
    # reg_forecast["Median_rental_price"] = future_price
    return reg_forecast

In [57]:
import warnings
warnings.filterwarnings('ignore')

##### DEPRICATED

In [126]:
forecast_bed_1_flat = get_forecast_all_suburbs(df_1bed_flat_no_covid.iloc[:,:163], yrs[:9], pred_cols, pop=False)
forecast_bed_2_flat = get_forecast_all_suburbs(bed_2_flat_no_covid.iloc[:,:163], yrs[:9], pred_cols, pop=False)
forecast_bed_2_house = get_forecast_all_suburbs(bed_2_house_no_covid.iloc[:,:163], yrs[:9], pred_cols, pop=False)
forecast_bed_3_flat = get_forecast_all_suburbs(bed_3_flat_no_covid.iloc[:,:163], yrs[:9], pred_cols, pop=False)
forecast_bed_3_house = get_forecast_all_suburbs(bed_3_house_no_covid.iloc[:,:163], yrs[:9], pred_cols, pop=False)
forecast_bed_4_house = get_forecast_all_suburbs(bed_4_house_no_covid.iloc[:,:163], yrs[:9], pred_cols, pop=False)

In [125]:
forecast_bed_4_house[bed_4_house.loc[0,'Suburb']]

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,year,Median_rental_price
0,37.6,1647.7,1011.2,333.4,2369.8,0.9,1.8,2022,1193.307877
1,37.7,1640.6,1053.8,333.3,2440.7,0.9,1.7,2023,1224.638326
2,37.8,1633.6,1096.3,333.1,2511.5,0.9,1.6,2024,1265.716854
3,37.9,1626.5,1138.9,333.0,2582.3,0.9,1.5,2025,1318.358274


################################################

#### Prediction including population data as well as the above

In [58]:
pred_cols = ['Median_Mar', 'Median_age_persons', 'Median_mortgage_repay_monthly', 'Median_tot_prsnl_inc_weekly', 
             'Median_rent_weekly', 'Median_tot_fam_inc_weekly','Average_num_psns_per_bedroom', 
             'Average_household_size', 'Pop', 'Alleged_Offender_Incident_Rate', 'Victimisation_Rate', 'Offence_Count']

In [59]:
def impute_missing(df, impute_value, pred_cols):
    for col in df:
        df[col] = df[col].replace("-",impute_value)
        for pred_col in pred_cols:
            if pred_col in col:
                df[col] = df[col].replace({'\$':''}, regex = True)
                df[col] = df[col].replace(",","", regex=True)
                df[col] = df[col].astype(float)
    return df

In [60]:
df_1bed_flat = impute_missing(df_1bed_flat, "-1", pred_cols)
bed_2_flat = impute_missing(bed_2_flat, "-1", pred_cols)
bed_2_house = impute_missing(bed_2_house, "-1", pred_cols)
bed_3_flat = impute_missing(bed_3_flat, "-1", pred_cols)
bed_3_house = impute_missing(bed_3_house, "-1", pred_cols)
bed_4_house = impute_missing(bed_4_house, "-1", pred_cols)

In [61]:
df_1bed_flat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Columns: 168 entries, Suburb to Offence_Count_2021
dtypes: float64(108), int64(1), object(59)
memory usage: 189.1+ KB


In [62]:
yrs = [2013,2014,2015,2016,2017,2018,2019, 2020, 2021]



df_1bed_flat_w_pop_cleaned = df_1bed_flat.dropna()
bed_2_flat_w_pop_cleaned = bed_2_flat.dropna()
bed_2_house_w_pop_cleaned = bed_2_house.dropna()
bed_3_flat_w_pop_cleaned = bed_3_flat.dropna()
bed_3_house_w_pop_cleaned = bed_3_house.dropna()
bed_4_house_w_pop_cleaned = bed_4_house.dropna()

In [63]:
bed_4_house_w_pop_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 0 to 108
Columns: 168 entries, Suburb to Offence_Count_2021
dtypes: float64(108), int64(1), object(59)
memory usage: 140.0+ KB


In [64]:
make_2D(bed_4_house_w_pop_cleaned.loc[0,:], yrs, pred_cols)

,Median_rental_price,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,Alleged_Offender_Incident_Rate,Victimisation_Rate,Offence_Count,year
0,1100.0,36.7,1711.611505,628.384025,334.550495,1732.435832,0.996667,2.893333,6943.0,3120.7,5785.3,11054.0,2013
1,1050.0,36.8,1704.515028,670.923608,334.421129,1803.258106,0.986667,2.773333,6963.0,3126.8,5318.3,11364.0,2014
2,1100.0,36.9,1697.418550,713.463190,334.291763,1874.080381,0.976667,2.653333,6982.0,3162.2,5530.1,11915.0,2015
3,1223.0,38.0,1650.618982,641.907382,363.136176,1868.867492,0.800000,3.000000,7002.0,3027.8,6083.2,12733.0,2016
4,1050.0,37.1,1683.225595,798.542354,334.033032,2015.724930,0.956667,2.413333,7022.0,2729.2,6388.3,13427.0,2017
5,1200.0,37.2,1676.129117,841.081936,333.903666,2086.547204,0.946667,2.293333,7042.0,2987.7,5620.4,12549.0,2018
6,1200.0,37.3,1669.032639,883.621518,333.774300,2157.369479,0.936667,2.173333,7061.0,2811.8,5442.8,12778.0,2019
7,1300.0,37.4,1661.936161,926.161100,333.644934,2228.191754,0.926667,2.053333,7081.0,2710.1,5662.4,12965.0,2020
8,1150.0,37.0,1674.691229,1025.748378,319.028679,2337.031610,1.000000,1.700000,7171.0,3851.3,5265.3,13408.0,2021


In [67]:
bed_4_house_w_pop_cleaned.loc[0,:]

Suburb                Albert Park-Middle Park-West St Kilda
Count_Mar2013                                            35
Median_Mar2013                                       1100.0
Count_Jun2013                                            40
Median_Jun2013                                       $1,125
                                      ...                  
Offence_Count_2017                                  13427.0
Offence_Count_2018                                  12549.0
Offence_Count_2019                                  12778.0
Offence_Count_2020                                  12965.0
Offence_Count_2021                                  13408.0
Name: 0, Length: 168, dtype: object

In [68]:
forecast_bed_1_flat_pop = get_forecast_all_suburbs(df_1bed_flat_w_pop_cleaned, yrs, pred_cols)
forecast_bed_2_flat_pop = get_forecast_all_suburbs(bed_2_flat_w_pop_cleaned, yrs, pred_cols)
forecast_bed_2_house_pop = get_forecast_all_suburbs(bed_2_house_w_pop_cleaned, yrs, pred_cols)
forecast_bed_3_flat_pop = get_forecast_all_suburbs(bed_3_flat_w_pop_cleaned, yrs, pred_cols)
forecast_bed_3_house_pop = get_forecast_all_suburbs(bed_3_house_w_pop_cleaned, yrs, pred_cols)
forecast_bed_4_house_pop = get_forecast_all_suburbs(bed_4_house_w_pop_cleaned, yrs, pred_cols)

In [71]:
forecast_bed_4_house_pop[bed_4_house.loc[0,'Suburb']]

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,Alleged_Offender_Incident_Rate,Victimisation_Rate,Offence_Count,year,Median_rental_price
0,37.4,1655.5,1033.4,327.8,2384.6,0.9,1.7,7151.7,3136.2,5536.9,13779.3,2022,1167.580369
1,37.5,1650.4,1081.7,326.2,2459.2,0.9,1.6,7176.1,3151.8,5508.8,14042.0,2023,1183.530602
2,37.5,1645.2,1129.9,324.6,2533.9,0.9,1.4,7200.5,3167.3,5480.7,14304.7,2024,1205.491029
3,37.6,1640.1,1178.2,323.0,2608.5,0.9,1.3,7224.9,3182.8,5452.7,14567.4,2025,1235.185924


#### Prediction without Covid

In [78]:
df_1bed_flat_no_covid = impute_missing(df_1bed_flat_no_covid, "-1", pred_cols)
bed_2_flat_no_covid = impute_missing(bed_2_flat_no_covid, "-1", pred_cols)
bed_2_house_no_covid = impute_missing(bed_2_house_no_covid, "-1", pred_cols)
bed_3_flat_no_covid = impute_missing(bed_3_flat_no_covid, "-1", pred_cols)
bed_3_house_no_covid = impute_missing(bed_3_house_no_covid, "-1", pred_cols)
bed_4_house_no_covid = impute_missing(bed_4_house_no_covid, "-1", pred_cols)

In [79]:
df_1bed_flat_no_covid_w_pop_cleaned = df_1bed_flat_no_covid.dropna()
bed_2_flat_no_covid_w_pop_cleaned = bed_2_flat_no_covid.dropna()
bed_2_house_no_covid_w_pop_cleaned = bed_2_house_no_covid.dropna()
bed_3_flat_no_covid_w_pop_cleaned = bed_3_flat_no_covid.dropna()
bed_3_house_no_covid_w_pop_cleaned = bed_3_house_no_covid.dropna()
bed_4_house_no_covid_w_pop_cleaned = bed_4_house_no_covid.dropna()

In [81]:
forecast_bed_1_flat_no_covid = get_forecast_all_suburbs(df_1bed_flat_no_covid_w_pop_cleaned, yrs[0:7], pred_cols)
forecast_bed_2_flat_no_covid = get_forecast_all_suburbs(bed_2_flat_no_covid_w_pop_cleaned, yrs[0:7], pred_cols)
forecast_bed_2_house_no_covid = get_forecast_all_suburbs(bed_2_house_no_covid_w_pop_cleaned, yrs[0:7], pred_cols)
forecast_bed_3_flat_no_covid = get_forecast_all_suburbs(bed_3_flat_no_covid_w_pop_cleaned, yrs[0:7], pred_cols)
forecast_bed_3_house_no_covid = get_forecast_all_suburbs(bed_3_house_no_covid_w_pop_cleaned, yrs[0:7], pred_cols)
forecast_bed_4_house_no_covid = get_forecast_all_suburbs(bed_4_house_no_covid_w_pop_cleaned, yrs[0:7], pred_cols)

In [84]:
forecast_bed_4_house_no_covid[bed_4_house.loc[0,'Suburb']]

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,Alleged_Offender_Incident_Rate,Victimisation_Rate,Offence_Count,year,Median_rental_price
0,37.7,1642.1,994.9,337.5,2359.0,0.9,1.9,7120.4,2644.2,5831.5,14200.1,2022,1161.849688
1,37.8,1635.0,1037.5,337.4,2429.8,0.9,1.8,7140.1,2585.7,5847.1,14523.5,2023,1193.163281
2,37.9,1627.9,1080.0,337.3,2500.6,0.9,1.6,7159.9,2527.2,5862.6,14846.9,2024,1241.804007
3,38.0,1620.8,1122.6,337.1,2571.4,0.9,1.5,7179.6,2468.7,5878.1,15170.2,2025,1296.712157


# Analysis

In [19]:
CORR_COLS = ['Median_rental_price', 'Median_age_persons', 'Median_mortgage_repay_monthly', 'Median_tot_prsnl_inc_weekly', 'Median_tot_fam_inc_weekly','Average_num_psns_per_bedroom', 
             'Average_household_size']

In [20]:
def get_corr_avg_med(raw_df, yrs):
    plot_df = pd.DataFrame()

    for feature in CORR_COLS:
        df = pd.DataFrame()
        for i, row in raw_df.iterrows():
            rw = make_2D(row, yrs)
            df[row["Suburb"]] = rw[feature]
        avg_df = df.mean()
        plot_df[feature] = avg_df.values
        
    return plot_df

In [21]:
df_1bed_flat_corr = get_corr_avg_med(df_1bed_flat, yrs)
bed_2_flat_corr = get_corr_avg_med(bed_2_flat, yrs)
bed_2_house_corr = get_corr_avg_med(bed_2_house, yrs)
bed_3_flat_corr = get_corr_avg_med(bed_3_flat, yrs)
bed_3_house_corr = get_corr_avg_med(bed_3_house, yrs)
bed_4_house_corr = get_corr_avg_med(bed_4_house, yrs)

In [23]:
df_1bed_flat_corr.corr()

,Median_rental_price,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size
Median_rental_price,1.000000,-0.562815,0.563109,0.452880,0.503312,0.413264,-0.193687
Median_age_persons,-0.562815,1.000000,-0.373593,-0.280401,-0.230330,-0.565155,0.294255
Median_mortgage_repay_monthly,0.563109,-0.373593,1.000000,0.749382,0.890527,0.498812,0.078648
Median_tot_prsnl_inc_weekly,0.452880,-0.280401,0.749382,1.000000,0.897067,0.340848,-0.151520
Median_tot_fam_inc_weekly,0.503312,-0.230330,0.890527,0.897067,1.000000,0.353864,0.034417
Average_num_psns_per_bedroom,0.413264,-0.565155,0.498812,0.340848,0.353864,1.000000,-0.127945
Average_household_size,-0.193687,0.294255,0.078648,-0.151520,0.034417,-0.127945,1.000000


In [ ]:
bed_2_house_corr[bed_2_house_corr['Median_rental_price'] == -1]

In [ ]:
plot_df[plot_df['Median_rental_price'] == -1]

In [ ]:
final_plot_df[final_plot_df['Median_rental_price'] == -1]

In [ ]:
df_1bed_flat_corr.corr().to_csv("../data/curated/corr/bed_1_flat_corr.csv")
bed_2_flat_corr.corr().to_csv("../data/curated/corr/bed_2_flat_corr.csv")
bed_2_house_corr.corr().to_csv("../data/curated/corr/bed_2_house_corr.csv")
bed_3_flat_corr.corr().to_csv("../data/curated/corr/bed_3_flat_corr.csv")
bed_3_house_corr.corr().to_csv("../data/curated/corr/bed_3_house_corr.csv")
bed_4_house_corr.corr().to_csv("../data/curated/corr/bed_4_house_corr.csv")

In [ ]:
avg_df = plot_df.iloc[:, :143].mean()
avg_df.sort_values(ascending = False)    

In [ ]:
avg_df.values

In [ ]:
sns.lineplot(plot_df.iloc[:, 133], yrs)

In [ ]:
sns.lineplot(x='year', y='value', hue='variable', data=pd.melt(plot_df, ['year']), legend=False)

In [ ]:
df_1bed_flat.sort_values("Median_Mar2012", ascending=False)

# Aggregate by Victoria

In [71]:
df_1bed_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_1_bed_flat_monetary_2011-2021.csv')
bed_2_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_flat_monetary_2011-2021.csv')
bed_2_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_house_monetary_2011-2021.csv')
bed_3_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_flat_monetary_2011-2021.csv')
bed_3_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_house_monetary_2011-2021.csv')
bed_4_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_4_house_monetary_2011-2021.csv')

In [72]:
df_1bed_flat_new = impute_missing(df_1bed_flat, np.nan)
bed_2_flat_new = impute_missing(bed_2_flat, np.nan)
bed_2_house_new = impute_missing(bed_2_house, np.nan)
bed_3_flat_new = impute_missing(bed_3_flat, np.nan)
bed_3_house_new = impute_missing(bed_3_house, np.nan)
bed_4_house_new = impute_missing(bed_4_house, np.nan)

In [69]:
bed_4_house

,Suburb,Count_Mar2011,Median_Mar2011,Count_Jun2011,Median_Jun2011,Count_Sep2011,Median_Sep2011,Count_Dec2011,Median_Dec2011,Count_Mar2012,...,Average_household_size_2014,Average_household_size_2015,Average_household_size_2016,Average_household_size_2017,Average_household_size_2018,Average_household_size_2019,Average_household_size_2020,Average_household_size_2021,Suburb.1,Postcode.1
0,Albert Park-Middle Park-West St Kilda,19,950.0,19,860.0,30,1100.0,34,1100.0,37,...,2.773333,2.653333,3.0,2.413333,2.293333,2.173333,2.053333,1.7,Scoresby,3182
1,Armadale,NaN,NaN,NaN,NaN,12,823.0,16,1100.0,15,...,2.640000,2.570000,2.8,2.430000,2.360000,2.290000,2.220000,2.0,Lilydale,3143
2,Carlton North,20,745.0,21,770.0,17,770.0,20,760.0,21,...,1.360000,1.330000,0.0,1.270000,1.240000,1.210000,1.180000,1.8,"Hotham Hill, North Melbourne",3053
3,Carlton-Parkville,37,775.0,36,766.0,35,750.0,31,750.0,30,...,2.706667,2.636667,2.8,2.496667,2.426667,2.356667,2.286667,2.1,"Attwood, Westmeadows",3052
4,CBD-St Kilda Rd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.773333,2.653333,3.0,2.413333,2.293333,2.173333,2.053333,1.7,Scoresby,3182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Torquay,121,440.0,120,450.0,126,450.0,143,450.0,160,...,2.560000,2.580000,2.6,2.620000,2.640000,2.660000,2.680000,2.7,Ocean Grove,3228
140,Traralgon,140,330.0,146,330.0,153,335.0,156,340.0,143,...,2.233333,2.233333,1.9,2.233333,2.233333,2.233333,2.233333,2.4,Nicholson,3844
141,Warragul,54,320.0,64,330.0,76,330.0,72,340.0,78,...,2.546667,2.556667,2.8,2.576667,2.586667,2.596667,2.606667,2.5,"Valencia Creek, Briagolong, Coongulla, Maffra,...",3820
142,Warrnambool,129,380.0,124,378.0,129,380.0,139,380.0,129,...,2.773333,2.703333,2.6,2.563333,2.493333,2.423333,2.353333,2.3,"Bushfield, Grassmere, Winslow, Woodford",3280


In [73]:
df_1bed_flat_new = df_1bed_flat_new.dropna()
bed_2_flat_new = bed_2_flat_new.dropna()
bed_2_house_new = bed_2_house_new.dropna()
bed_3_flat_new = bed_3_flat_new.dropna()
bed_3_house_new = bed_3_house_new.dropna()
bed_4_house_new = bed_4_house_new.dropna()

In [78]:
bed_3_flat_new

,Suburb,Count_Mar2011,Median_Mar2011,Count_Jun2011,Median_Jun2011,Count_Sep2011,Median_Sep2011,Count_Dec2011,Median_Dec2011,Count_Mar2012,...,Average_household_size_2014,Average_household_size_2015,Average_household_size_2016,Average_household_size_2017,Average_household_size_2018,Average_household_size_2019,Average_household_size_2020,Average_household_size_2021,Suburb.1,Postcode.1
0,Albert Park-Middle Park-West St Kilda,20,595.0,24,595.0,23,625.0,22,610.0,27,...,2.773333,2.653333,3.0,2.413333,2.293333,2.173333,2.053333,1.7,Scoresby,3182
1,Armadale,33,530.0,33,550.0,38,575.0,43,600.0,38,...,2.640000,2.570000,2.8,2.430000,2.360000,2.290000,2.220000,2.0,Lilydale,3143
2,Carlton North,21,590.0,18,595.0,13,600.0,14,600.0,11,...,1.360000,1.330000,0.0,1.270000,1.240000,1.210000,1.180000,1.8,"Hotham Hill, North Melbourne",3053
3,Carlton-Parkville,82,556.0,88,550.0,92,553.0,87,560.0,85,...,2.706667,2.636667,2.8,2.496667,2.426667,2.356667,2.286667,2.1,"Attwood, Westmeadows",3052
4,CBD-St Kilda Rd,323,680.0,303,695.0,287,700.0,279,690.0,271,...,2.773333,2.653333,3.0,2.413333,2.293333,2.173333,2.053333,1.7,Scoresby,3182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Torquay,48,368.0,49,365.0,55,365.0,55,370.0,49,...,2.560000,2.580000,2.6,2.620000,2.640000,2.660000,2.680000,2.7,Ocean Grove,3228
140,Traralgon,26,255.0,23,255.0,25,260.0,38,278.0,47,...,2.233333,2.233333,1.9,2.233333,2.233333,2.233333,2.233333,2.4,Nicholson,3844
141,Warragul,14,230.0,17,230.0,16,235.0,17,230.0,16,...,2.546667,2.556667,2.8,2.576667,2.586667,2.596667,2.606667,2.5,"Valencia Creek, Briagolong, Coongulla, Maffra,...",3820
142,Warrnambool,46,290.0,54,300.0,58,300.0,57,300.0,73,...,2.773333,2.703333,2.6,2.563333,2.493333,2.423333,2.353333,2.3,"Bushfield, Grassmere, Winslow, Woodford",3280


In [80]:
def get_avg_by_year(df, yrs):
    avg = {}
    for i,rw in df.iterrows():
        X_df = make_2D(rw, yrs)
        for j,row in X_df.iterrows():
            if row['year'] in avg.keys():
                avg[row['year']] += row['Median_rental_price']
            else:
                avg[row['year']] = row['Median_rental_price']
           
    avg = {x: avg[x]/df.shape[0] for x in avg}
    return avg
    

In [81]:
df_1bed_flat_avg_by_yr = get_avg_by_year(df_1bed_flat_new, yrs)
bed_2_flat_avg_by_yr = get_avg_by_year(bed_2_flat_new, yrs)
bed_2_house_avg_by_yr = get_avg_by_year(bed_2_house_new, yrs)
bed_3_flat_avg_by_yr = get_avg_by_year(bed_3_flat_new, yrs)
bed_3_house_avg_by_yr = get_avg_by_year(bed_3_house_new, yrs)
bed_4_house_avg_by_yr = get_avg_by_year(bed_4_house_new, yrs)


In [90]:
bed_4_house_avg_by_yr

{2011.0: 467.4307692307692,
 2012.0: 494.5692307692308,
 2013.0: 495.53076923076924,
 2014.0: 512.9538461538461,
 2015.0: 530.9153846153846,
 2016.0: 541.4923076923077,
 2017.0: 559.3,
 2018.0: 576.4615384615385,
 2019.0: 592.0153846153846,
 2020.0: 603.1461538461539,
 2021.0: 596.7307692307693}